# Custom CNN with Keras - GPU

- I built this custom CNN using Keras
- The model was trained on AMD GPU with PlaidML backend. I did not use TensorFlow since i do not have an Nvidia GPU, but that can be changed by setting `environ["KERAS_BACKEND"] = "tensorflow"`
- This model can classify dogs and cats

## Import all libraries and set backend to PlaidML

To use PlaidML as backend follow these steps:
- activate project environment
- `plaidml-setup`
- `Enable experimental device support? (y,n)[n]:n`
- `Please choose a default device: 3` most probably 1 would be your CPU, 2 would be the integrated GPU of your CPU and 3 would be your dedicated GPU
- `Save settings to /Users/veersingh/.plaidml? (y,n)[y]:y`

In [1]:
# import libraries and set backend to plaidML
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import json
import PIL

Using plaidml.keras.backend backend.


## Preprocessing the data

- First we need to rescale the pixel values. Currently the pixel values are in the range [0,255] since we are using 8 bit color or 2^8 values per pixel. We can just divide each pixel value with 255 and that will rescale all pixels to the range [0,1]. This reduces computation.
- Next we need to label and resize the image since the model accepts images of the fixed size 100x100x3 where the image is 100 pixels in length and width and has 3 channels RGB. We can do this automatically by using ImageDataGenerator which has the method flow_from_director.

In [2]:
# Specify the paths
train_dataset_path = '/Users/veersingh/Desktop/cats_dogs/train'
valid_dataset_path = '/Users/veersingh/Desktop/cats_dogs/valid'

In [3]:
# Rescale
train = ImageDataGenerator(rescale=1 / 255)
valid = ImageDataGenerator(rescale=1 / 255)

# Define train and valid dataset
# This method is useful since our images are in their respective class folders

# seed = random number to maintain same output everytime
# directory = path of train or valid dataset
# target_size = resize the input image to required shape
# batch_size = default batch size of 32
# class_mode = categorical for multiclass
# color_mode = depending on number of channels required by model, grayscale for 1 channel, rgb for 3 channel

train_dataset = train.flow_from_directory(seed=1,
                                          directory=train_dataset_path,
                                          target_size=(100, 100),
                                          batch_size=32,
                                          class_mode = 'binary',
                                          color_mode='rgb')

valid_dataset = valid.flow_from_directory(seed=1,
                                         directory=valid_dataset_path,
                                         target_size=(100, 100),
                                         batch_size=32,
                                         class_mode = 'binary',
                                         color_mode='rgb')

# print labels
print('\n')
print(f'Label for cat -> {train_dataset.class_indices["cat"]}')
print(f'Label for dog -> {train_dataset.class_indices["dog"]}')

Found 16000 images belonging to 2 classes.
Found 7000 images belonging to 2 classes.


Label for cat -> 0
Label for dog -> 1


## Create the Model

In [4]:
# Model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu',input_shape=(100, 100, 3)))

# Maxpool layer 1
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 3
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 4
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))

# Maxpool layer 4
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten to 1D
model.add(Flatten())

# Fully connected layer with 512 neurons
model.add(Dense(units=512, activation='relu'))

# Output layer with single neuron which gives 0 for cat and 1 for dog
model.add(Dense(units=1, activation='sigmoid'))

INFO:plaidml:Opening device "metal_amd_radeon_pro_5500m.0"


In [5]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         147584    
__________

## Train and Save the Model

In [7]:
# Train the model

# start the timer
start_time = time.time()

# generator = input training data created using ImageDataGenerator
# steps_per_epoch = number of total training images/batch size = 16000/32 = 500
# epochs = number of times we go through the entire training dataset
# validation_data = validation dataset
# verbose = it will print updates on every epoch

history = model.fit_generator(generator=train_dataset,
                              steps_per_epoch=500,
                              epochs=30,
                              validation_data=valid_dataset,
                              verbose=2)

# save the model
model.save('models/cats_dogs_trained_model.h5')

# stop the timer
time_elapsed = round(time.time()-start_time, 2)

print(f'\n\nThe model took {time_elapsed} seconds to train' )

Epoch 1/30


/Users/veersingh/opt/anaconda3/envs/lenet_alexnet/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:811: UserWarning: Truncated File Read
  warnings.warn(str(msg))


 - 95s - loss: 0.6700 - acc: 0.5806 - val_loss: 0.6355 - val_acc: 0.6499
Epoch 2/30
 - 95s - loss: 0.5947 - acc: 0.6809 - val_loss: 0.5511 - val_acc: 0.7096
Epoch 3/30
 - 95s - loss: 0.5479 - acc: 0.7234 - val_loss: 0.5277 - val_acc: 0.7431
Epoch 4/30
 - 90s - loss: 0.4975 - acc: 0.7556 - val_loss: 0.4668 - val_acc: 0.7804
Epoch 5/30
 - 91s - loss: 0.4343 - acc: 0.7984 - val_loss: 0.4170 - val_acc: 0.8104
Epoch 6/30
 - 91s - loss: 0.3877 - acc: 0.8211 - val_loss: 0.4588 - val_acc: 0.7809
Epoch 7/30
 - 90s - loss: 0.3800 - acc: 0.8287 - val_loss: 0.4133 - val_acc: 0.8086
Epoch 8/30
 - 91s - loss: 0.3149 - acc: 0.8614 - val_loss: 0.4178 - val_acc: 0.8156
Epoch 9/30
 - 91s - loss: 0.2938 - acc: 0.8701 - val_loss: 0.3818 - val_acc: 0.8329
Epoch 10/30
 - 90s - loss: 0.2437 - acc: 0.8964 - val_loss: 0.3679 - val_acc: 0.8446
Epoch 11/30
 - 90s - loss: 0.2072 - acc: 0.9116 - val_loss: 0.3864 - val_acc: 0.8411
Epoch 12/30
 - 90s - loss: 0.1625 - acc: 0.9349 - val_loss: 0.4612 - val_acc: 0.8191


## Create log to plot graphs and serialize to JSON

In [8]:
log = history.history

# add time to train to log
log['time_to_train_gpu'] = time_elapsed


with open("log_gpu.json", "w") as data_file:
    json.dump(log, data_file, indent=4)